<a href="https://colab.research.google.com/github/aninsung/Machine-Learning-Programming/blob/main/4%EC%A3%BC%EC%B0%A8_mhealth_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## (오늘 실습과제)

Mhealth dataset으로 인간행동 분류하기

일단 데이터 읽어서 subject 번호가 컬럼면에 포함된 통합 data frame 만들고 CNN, LSTM, CNN-LSTM 해보기.

1. LLM(sLLM)을 제외한 어떤 모델을 써도 가능. 단 라이브러리 import 불가. 모델의 풀 소스가 있어야 함.

 2. 10명을 1+2, 3+4, 4+5,.... 5개그룹으로 나눠 2명씩 테스트 데이터로 하면서 cross validation.

 3. 데이터 증대는 안됨.

4. 오직 모델로만 성능 향상

5. 상위 30%씩 차등.

6. 성적에 반영

7. 11월15일까지. Github게시

교차 평가로 평균 F1 기준.

8. 본인이 완벽하게 소스 및 모델을 설명할 수 있어야 함. GPT를 쓰던 뭘 쓰던 그건 상관안함. 재현성이 보장되어야 함.
--------------------------------------------------------------------------
--------------------------------------------------------------------------

## 데이터설명
📌 Dataset Summary

Subjects: 10명 (다양한 프로필의 자원자)

Activities: 12개

Sensors: 3개 (가슴, 오른쪽 손목, 왼쪽 발목 → elastic strap으로 고정)

측정 정보:

Acceleration (가속도)

Gyroscope (회전율)

Magnetometer (자기장 방향)

Chest sensor: 2-lead ECG 추가 (활동 인식에는 사용 안 함, 향후 연구용)

Sampling Rate: 50 Hz

환경: Out-of-lab, 제약 없이 자유롭게 활동 수행 (단, 최선을 다해 실행)

📌 Activity Set (12 classes)

정적 활동: Standing still, Sitting, Lying down

일상/이동 활동: Walking, Climbing stairs, Cycling

운동/역동적 활동: Jogging, Running, Jump front & back

반복 동작: Waist bends, Arm elevation, Knees bending

📌 특징

ECG 포함: 운동 중 심전도 기록 → 부정맥, 심박수 모니터링, 운동 효과 분석 가능

다양성 확보:

동작 강도 차이 (예: Sitting vs. Running)

속도 차이 (예: Walking vs. Jogging)

다른 신체 부위 동작 (예: Arm elevation vs. Knees bending)

실생활 반영: 실제 환경에서 수행되어 일반화 성능 높음

In [33]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, LSTM, TimeDistributed, Dropout
from sklearn.preprocessing import StandardScaler
from scipy import stats
import os


DATASET_BASE_PATH = "/content/drive/MyDrive/Colab Notebooks/mhealth+dataset/MHEALTHDATASET"

In [34]:
# 10개 주제의 모든 데이터를 로드하여 하나의 데이터프레임으로 통합
all_dfs = []
column_names = [
    'chest_acc_x', 'chest_acc_y', 'chest_acc_z', 'ecg_lead1', 'ecg_lead2', 'left_ankle_acc_x',
    'left_ankle_acc_y', 'left_ankle_acc_z', 'left_ankle_gyro_x', 'left_ankle_gyro_y',
    'left_ankle_gyro_z', 'left_ankle_mag_x', 'left_ankle_mag_y', 'left_ankle_mag_z',
    'right_wrist_acc_x', 'right_wrist_acc_y', 'right_wrist_acc_z', 'right_wrist_gyro_x',
    'right_wrist_gyro_y', 'right_wrist_gyro_z', 'right_wrist_mag_x', 'right_wrist_mag_y',
    'right_wrist_mag_z', 'activity_label'
]

for i in range(1, 11):
    file_path = os.path.join(DATASET_BASE_PATH, f"mHealth_subject{i}.log")
    try:
        df = pd.read_csv(file_path, sep='\s+', header=None, names=column_names)
        df['subject'] = i
        all_dfs.append(df)
    except FileNotFoundError:
        print(f"오류: {file_path}를 찾을 수 없습니다.")
        all_dfs = []
        break

if all_dfs:
    mhealth_df = pd.concat(all_dfs, ignore_index=True)
    print("데이터 로드 및 통합 완료.")
    print(f"전체 데이터 형태: {mhealth_df.shape}")

<>:14: SyntaxWarning: invalid escape sequence '\s'
<>:14: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-937467728.py:14: SyntaxWarning: invalid escape sequence '\s'
  df = pd.read_csv(file_path, sep='\s+', header=None, names=column_names)


In [35]:
# 5-겹 교차검증을 위해 피험자 기준으로 데이터를 나누고 스케일링
scaled_data_folds = []
if 'mhealth_df' in locals():
    feature_cols = mhealth_df.columns.drop(['activity_label', 'subject'])
    label_col = 'activity_label'
    num_classes = mhealth_df[label_col].nunique()
    test_subject_pairs = [(i, i + 1) for i in range(1, 11, 2)]

    for test_subjects in test_subject_pairs:
        train_df = mhealth_df[~mhealth_df['subject'].isin(test_subjects)].copy()
        test_df = mhealth_df[mhealth_df['subject'].isin(test_subjects)].copy()

        scaler = StandardScaler()
        train_df.loc[:, feature_cols] = scaler.fit_transform(train_df[feature_cols])
        test_df.loc[:, feature_cols] = scaler.transform(test_df[feature_cols])

        scaled_data_folds.append({'train_df': train_df, 'test_df': test_df})

    print(f"총 {len(scaled_data_folds)}개 폴드의 데이터 분할 및 스케일링 완료.")


## 통합 데이터프레임 정보:
<class 'pandas.core.frame.DataFrame'>
Index: 343195 entries, 6656 to 1213645
Data columns (total 23 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   chest_acc_x         343195 non-null  float64
 1   chest_acc_y         343195 non-null  float64
 2   chest_acc_z         343195 non-null  float64
 3   left_ankle_acc_x    343195 non-null  float64
 4   left_ankle_acc_y    343195 non-null  float64
 5   left_ankle_acc_z    343195 non-null  float64
 6   left_ankle_gyro_x   343195 non-null  float64
 7   left_ankle_gyro_y   343195 non-null  float64
 8   left_ankle_gyro_z   343195 non-null  float64
 9   left_ankle_mag_x    343195 non-null  float64
 10  left_ankle_mag_y    343195 non-null  float64
 11  left_ankle_mag_z    343195 non-null  float64
 12  right_wrist_acc_x   343195 non-null  float64
 13  right_wrist_acc_y   343195 non-null  float64
 14  right_wrist_acc_z   343195 non-null  float64
 15  right_wrist_gyro_

In [36]:
# 스케일링된 데이터를 윈도우 형태로 변환하여 최종 학습 데이터셋 생성
prepared_data = []
if 'scaled_data_folds' in locals():
    feature_cols = mhealth_df.columns.drop(['activity_label', 'subject'])
    label_col = 'activity_label'

    for fold in scaled_data_folds:
        train_df = fold['train_df']
        test_df = fold['test_df']

        X_train, y_train = [], []
        for i in range(0, len(train_df) - 128, 64):
            X_train.append(train_df[feature_cols].iloc[i: i + 128].values)
            y_train.append(stats.mode(train_df[label_col].iloc[i: i + 128], keepdims=True)[0][0])

        X_test, y_test = [], []
        for i in range(0, len(test_df) - 128, 64):
            X_test.append(test_df[feature_cols].iloc[i: i + 128].values)
            y_test.append(stats.mode(test_df[label_col].iloc[i: i + 128], keepdims=True)[0][0])

        prepared_data.append({
            'X_train': np.array(X_train), 'y_train': np.array(y_train),
            'X_test': np.array(X_test), 'y_test': np.array(y_test)
        })

    print(f"총 {len(prepared_data)}개 폴드의 윈도우 생성 완료.")

In [37]:
# 생성된 데이터의 형태 확인
if prepared_data:
    X_train_sample = prepared_data[0]['X_train']
    y_train_sample = prepared_data[0]['y_train']

    print("X_train shape:", X_train_sample.shape)
    print("y_train shape:", y_train_sample.shape)

In [ ]:
def create_windows(data, feature_cols, label_col):
    X, y = [], []
    for i in range(0, len(data) - 128, 64):
        # 특징 데이터 추출
        X.append(data[feature_cols].iloc[i: i + 128].values)
        # 윈도우 내에서 가장 빈번한 값으로 레이블 결정
        y.append(stats.mode(data[label_col].iloc[i: i + 128], keepdims=True)[0][0])
    return np.array(X), np.array(y)


===== Fold 1/5 (Test Subjects: (1, 2)) =====


In [ ]:
if prepared_data:
    accuracies = []
    for i, fold_data in enumerate(prepared_data):
        X_train, y_train = fold_data['X_train'], fold_data['y_train']
        X_test, y_test = fold_data['X_test'], fold_data['y_test']

        input_shape = (X_train.shape[1], X_train.shape[2])

        model = Sequential([
            LSTM(100, input_shape=input_shape),
            Dropout(0.5),
            Dense(100, activation='relu'),
            Dense(num_classes, activation='softmax')
        ])
        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

        model.fit(X_train, y_train, epochs=20, batch_size=64, verbose=0)
        loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
        accuracies.append(accuracy)
        print(f"LSTM | Fold {i+1}/5 | Accuracy: {accuracy:.4f}")

    final_accuracies['lstm'] = np.mean(accuracies)

In [ ]:
if prepared_data:
    accuracies = []
    for i, fold_data in enumerate(prepared_data):
        X_train, y_train = fold_data['X_train'], fold_data['y_train']
        X_test, y_test = fold_data['X_test'], fold_data['y_test']

        X_train_reshaped = X_train.reshape((X_train.shape[0], 1, X_train.shape[1], X_train.shape[2]))
        X_test_reshaped = X_test.reshape((X_test.shape[0], 1, X_test.shape[1], X_test.shape[2]))

        input_shape = (None, X_train.shape[1], X_train.shape[2])

        model = Sequential([
            TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'), input_shape=input_shape),
            TimeDistributed(MaxPooling1D(pool_size=2)),
            TimeDistributed(Flatten()),
            LSTM(100),
            Dropout(0.5),
            Dense(100, activation='relu'),
            Dense(num_classes, activation='softmax')
        ])
        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

        model.fit(X_train_reshaped, y_train, epochs=20, batch_size=64, verbose=0)
        loss, accuracy = model.evaluate(X_test_reshaped, y_test, verbose=0)
        accuracies.append(accuracy)
        print(f"CNN-LSTM | Fold {i+1}/5 | Accuracy: {accuracy:.4f}")

    final_accuracies['cnn-lstm'] = np.mean(accuracies)

In [ ]:
print("\n===== 최종 모델 평균 정확도 비교 =====")
if 'final_accuracies' in locals() and final_accuracies:
    for model_name, avg_accuracy in final_accuracies.items():
        print(f"{model_name.upper():<10} : {avg_accuracy:.4f}")
else:
    print("훈련된 모델이 없습니다.")